Quiz: Multi Layer Perceptron
==================

In this quiz you are asked to modify the MLP model used in the tutorial in order to train the network on the [Iris flower dataset](../iris/iris.ipynb). The dataset is already provided in the TFRecord format and you can find it [here](../iris). In the snippets below you will find an **hashtag** `#QUIZ` in the parts where you are asked to complete the code. Remember that the Tensorflow official documentation is your best friend when you cannot find a specific method.

Defining the model
---------------------

Here you have to define the MLP model for the new dataset. What you are asked to do is:

- Define the model. The dataset has **4 input features** (sepal length, sepal width, petal length, petal width) meaning that your MLP must have 4 input units. Morevoer the dataset has **3 classes** (0=Setosa, 1=Versicolor, 2=Virginica) meaning that your perceptron must have 3 output units.

- Design a new **accuracy metric**. In the XOR example there were only two possible classes, whereas here there are three. You should use the `argmax()` method in tensorflow to get the class with higher probability returned from the MLP. Then you must compare it with the actual target class that is stored in a one-hot vector.

In [ ]:
def my_model_fn(features, labels, mode):
    #QUIZ: here you must define the model for the new dataset 

        
    #PREDICT mode
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {"classes": tf.round(y),
                       "probabilities": y}
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    #TRAIN mode
    elif mode == tf.estimator.ModeKeys.TRAIN:
        loss = tf.losses.mean_squared_error(labels=labels, predictions=y)
        #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        optimizer = tf.train.AdamOptimizer()
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        #QUIZ: here you must define a new accuracy metric.
        #accuracy = tf.metrics.accuracy(???)
        
        tf.summary.scalar('accuracy', accuracy[1]) #<-- accuracy[1] to grab the value
        logging_hook = tf.train.LoggingTensorHook({"accuracy" : accuracy[1]}, every_n_iter=250)
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op, training_hooks =[logging_hook])
    #EVAL mode
    elif mode == tf.estimator.ModeKeys.EVAL:
        loss = tf.losses.mean_squared_error(labels=labels, predictions=y)
        #QUIZ: here you must define a new accuracy metric.      
        #accuracy = tf.metrics.accuracy(???)
        
        eval_metric = {"accuracy": accuracy}
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric)

In [ ]:
mlp = tf.estimator.Estimator(model_fn=my_model_fn, model_dir="./tf_mlp_model")

Train the model
------------------

In this section you have to define a dataset object and an iterator. The parse function is already implemented. The TFRecord files for the Iris dataset are ready to be used and included in this repository. Here you have to declare a Tensorflow dataset and an iterator. I suggest you to read the official documentation of Tensorflow for knowning more about those classes. 

In [ ]:
def my_input_fn():  
    def _parse_function(example_proto):
        features = {"feature": tf.VarLenFeature(tf.float32),
                    "label": tf.FixedLenFeature((), tf.int64, default_value=0)}
        parsed_features = tf.parse_single_example(example_proto, features)
        feature = tf.cast(parsed_features["feature"], tf.float32)
        feature = tf.sparse_tensor_to_dense(feature, default_value=0)
        label_one_hot = tf.one_hot(parsed_features["label"], depth=3)
        return feature, label_one_hot

    #QUIZ: load the TFRecord dataset using the method tf.data.TFRecordDataset()
    
    #QUIZ: parse the dataset using the map() method pointing to the _parse_function() above

    #QUIZ: store the dataset in cache
    
    #QUIZ: shuffle the dataset using the shuffle() method (be carefull with the buffer_size parameter)

    #QUIZ: use the method repeat() to decide how many time you would like to use the dataset

    #QUIZ: define the batch size through the batch() method

    #QUIZ: make a one-shot iterator
    
    #QUIZ: get new values using get_next()
    
    return batch_features, batch_labels

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)

In [ ]:
mlp.train(input_fn=my_input_fn, steps=5000)

Test the model
------------------

Now you have to test the model on the test set. You have to declare a new input function, a dataset and an iterator. You can reuse the code above but you must be careful with the `batch()` and `repeat()` methods.

In [ ]:
def my_input_fn():
        #QUIZ: load and parse the test set. This part is very similar to the function
        #you already implemented above. 
    
    
        return batch_features, batch_labels

Improve the performances
-----------------------------

Now you have to modify some critical parameters in your model in order to improve the performances. I suggest you  to play with the number of hidden units. You can try to increase the number of units and use **dropout** to avoid overfitting. Moreover you can use a different optimizer. Good optimizers are the RMSProp, Adam, and Adagrad. However, sometimes nothing is better than a well-tuned stocastich gradient descent with decaying learning rate. Good luck...